In [76]:
import os
import pdfplumber
import pandas as pd
import numpy as np
import re

# libro = pd.read_excel('file_informeMensualREC - 2025-12-02T162142.258.xlsx', skiprows=5)
# libro

headers_softland = ['cuenta SOFTLAND', 'debe', 'haber', 'glosa', '', '', '', '' ,'', '', '', '', '', '', '', 'ceco', '', '', 'rut', 'tipo doc', 'N° DOC', 'FECHA', 'FECHA', 'tipo doc', 'N° DOC']

In [77]:
ruta_carpeta = 'boletas_pdf/'
df_glosa = pd.DataFrame()
temp_glosa = pd.DataFrame()

for file in os.listdir(ruta_carpeta):
    ruta_pdf = ruta_carpeta + file

    with pdfplumber.open(ruta_pdf) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
        lines = text.split('\n')


    id_boleta = 'N ° '
    for linea in lines:
        if id_boleta in linea:
            id_boleta_valor = linea.split(id_boleta)[1].strip()
            print(f'ID Boleta: {id_boleta_valor}')

    id_rut = 'RUT: '
    for linea in lines:
        if id_rut in linea:
            id_rut_valor = linea.split(id_rut)[1].strip()
            print(f'RUT: {id_rut_valor}')

    id_fecha = 'Hora Emisión: '
    for linea in lines:
        if id_fecha in linea:
            id_fecha_valor = linea.split(id_fecha)[1].strip()
            id_fecha_valor = id_fecha_valor[:-6]
            print(f'Fecha: {id_fecha_valor}')

    #Glosas profesionales
    ini = 'Por atención profesional:'
    fin = 'Total Honorarios'
    flag = False

    for linea in lines:
        if ini in linea:
            flag = True
            continue
        
        if fin in linea:
            flag = False
            continue
        
        if flag == True:
            glosa = linea.strip()
            
            valor_ = glosa.split()[-1]
            if '.' in valor_:
                valor = np.int64(valor_.replace('.', ''))
                glosa = glosa.replace(valor_, '')
            else:
                valor = None


    #Buscar total e impuesto retenido
    ini = 'Total Honorarios'
    flag = False
    data = []
    temp_cont = 0

    for linea in lines:
        if ini in linea:
            flag = True
               
        if flag == True:
            temp_cont += 1
            temp = linea
            if temp_cont == 2 and 'Retenido' not in temp:
                data.append(np.int64('0'))
                data.append(np.int64(xtotal))
                break
            else:
                xtotal = temp.split()[-1].replace('.', '')
                data.append(np.int64(xtotal))

        if temp_cont == 3:
            break
    print(data)

    temp_glosa = pd.DataFrame({'Numero Boleta': [id_boleta_valor], 'RUT': [id_rut_valor], 'Glosa': [glosa], 'Fecha': [id_fecha_valor], 'Valor':[valor], '01_Total Honorarios': [data[0]], '02_Impuesto Retenido': [data[1]], '03_Total': [data[2]]})

    #Unir glosas partidas en dos lineas
    df_glosa = df_glosa.reset_index(drop=True)
    for ix in df_glosa.index:
        if df_glosa.at[ix, 'Valor'] == None:
            df_glosa.at[ix-1, 'Glosa'] = df_glosa.at[ix-1, 'Glosa'] + ' ' + df_glosa.at[ix, 'Glosa']
            df_glosa = df_glosa.drop(ix)
    
    df_glosa = pd.concat([df_glosa, temp_glosa], ignore_index=True)


#Buscar la OC
df_glosa['OC'] = ''
for ix in df_glosa.index:
    temp = df_glosa.at[ix, 'Glosa']
    oc_match = re.search(r'OC[:\s]*([A-Za-z0-9-]+)', temp)
    if oc_match:
        df_glosa.at[ix, 'OC'] = oc_match.group(1)
        continue

    oc_match = re.search(r'ORDEN DE COMPRA[:\s]*([A-Za-z0-9-]+)', temp)
    if oc_match:
        df_glosa.at[ix, 'OC'] = oc_match.group(1)
        continue

    oc_match = re.search(r'ORDEN COMPRA[:\s]*([A-Za-z0-9-]+)', temp)
    if oc_match:
        df_glosa.at[ix, 'OC'] = oc_match.group(1)
        continue

    oc_match = re.search(r'ORDEN[:\s]*([A-Za-z0-9-]+)', temp)
    if oc_match:
        df_glosa.at[ix, 'OC'] = oc_match.group(1)
        continue


#Aplicar OC a todas las filas de la misma boleta
for boleta in df_glosa['Numero Boleta'].unique():
    try:
        oc_valor = df_glosa.loc[(df_glosa['Numero Boleta'] == boleta) & (df_glosa['OC'] != ''), 'OC'].values[0]
        df_glosa.loc[df_glosa['Numero Boleta'] == boleta, 'OC'] = oc_valor
    except:
        pass
df_glosa.loc[df_glosa['OC'] != '', 'OC'] = 'OC' + df_glosa['OC']
df_glosa.loc[df_glosa['OC'] == '', 'OC'] = 'S/OC'

df_glosa['RUT_NUM'] = df_glosa['RUT'].apply(lambda x: x.replace('.', '').split('−')[0])
df_glosa['new_glosa'] = df_glosa['RUT_NUM'] + '-' + df_glosa['OC'] + '-BH' + df_glosa['Numero Boleta'] + ' ' + df_glosa['Glosa'] + ' - CPM'



#Acomodas glosas a formato Softland

formato = df_glosa.copy()
formato['Cuenta SOFTLAND'] = ''
formato = formato.drop_duplicates(subset=['Numero Boleta'], keep='first')
formato = formato[['Fecha', 'RUT_NUM', 'Numero Boleta', 'Cuenta SOFTLAND', 'new_glosa', '01_Total Honorarios', '02_Impuesto Retenido', '03_Total']]
formato = pd.melt(formato, id_vars=['Fecha', 'RUT_NUM', 'Numero Boleta', 'Cuenta SOFTLAND', 'new_glosa'], value_vars=['01_Total Honorarios', '02_Impuesto Retenido', '03_Total'], var_name='Descripcion', value_name='Valor')
formato.columns = ['Fecha', 'RUT_NUM', 'Numero Boleta', 'Cuenta SOFTLAND', 'Glosa', 'Descripcion', 'Valor']
formato['DEBE'] = 0
formato['HABER'] = 0
formato.loc[formato['Descripcion'] == '01_Total Honorarios', 'DEBE'] = formato['Valor']
formato.loc[formato['Descripcion'] == '02_Impuesto Retenido', 'HABER'] = formato['Valor']
formato.loc[formato['Descripcion'] == '03_Total', 'HABER'] = formato['Valor']
formato = formato.sort_values(by=['Numero Boleta', 'Descripcion'])


#necesito generar 11 columnas vacias, con nombre vacio
for x in range(1, 13+1):
    formato[f'borrar_{x}'] = ''


#Acomodar columnas faltantes a formato softland
formato['rut'] = formato['RUT_NUM']
formato['ceco'] = ''
formato['tipo doc_1'] = 'BH'
formato['tipo doc_2'] = 'BH'
formato['N° DOC_1'] = formato['Numero Boleta']
formato['N° DOC_2'] = formato['Numero Boleta']
formato['FECHA_1'] = formato['Fecha']
formato['FECHA_2'] = formato['Fecha']


formato = formato[['Cuenta SOFTLAND', 'DEBE', 'HABER', 'Glosa', 'borrar_1', 'borrar_2', 'borrar_3', 'borrar_4', 'borrar_5', 'borrar_6', 'borrar_7', 'borrar_8', 'borrar_9', 'borrar_10', 'borrar_11', 'ceco', 'borrar_12', 'borrar_13', 'rut', 'tipo doc_1', 'N° DOC_1', 'FECHA_1', 'FECHA_2', 'tipo doc_2', 'N° DOC_2']]
formato.columns = headers_softland

formato.to_excel('formato_softland.xlsx', index=False)




ID Boleta: 8
RUT: 20.258.337−7
Fecha: 23/06/2025
[np.int64(209795), np.int64(30420), np.int64(179375)]
ID Boleta: 152666
RUT: 5.248.026−4
Fecha: 21/11/2025
[np.int64(13500), np.int64(0), np.int64(13500)]
ID Boleta: 30
RUT: 8.838.991−3
Fecha: 14/11/2025
[np.int64(1486502), np.int64(215543), np.int64(1270959)]
ID Boleta: 278
RUT: 10.689.553−8
Fecha: 27/11/2025
[np.int64(1040000), np.int64(182000), np.int64(858000)]
